Please send your solution to carlop@ethz.ch

Construct a 2x2x2 supercell of the orthorombic phase of MAPbI3 with parameters taken from the Supplementary information of: https://doi.org/10.1039/C4CC09944C
<ul>
<li>Starting from the supercell create 20 different atomistic models (or just 5 to save time) each of them containing 10 $H_2O$ molecules.</li>
<li>Each water  molecule should be in a distance range between $1.5A$ and $2.5A$ from a N atom of the methylammonium molecules. </li> 
<li>The minimum distance between the water molecules and any other atom of the cell should be $\geq 1.5A$</li>
</ul>

In [1]:
#pip install spglib --user
#pip install nglview --user

In [1]:
import numpy as np
from numpy.linalg import norm

from ase.io import read,write
from ase.visualize import view,ngl
from ase.spacegroup import crystal
from ase.spacegroup import Spacegroup
from ase.data import atomic_numbers, atomic_names
from ase import Atoms
from ase import neighborlist
from itertools import product

from scipy.spatial.transform import Rotation as R

import nglview
from ase.build import molecule

In [2]:
#there are four inequivalent atoms with crystal coordinates:
#ICSD structure 428898 (1788472.cif)
MAPbI3=crystal(
    symbols=['Pb','I','I','N','C','H','H','H','H'],
    basis=[(0.5,0,0),
           (0.4842,0.25,-0.0562),
           (0.1886, 0.0147, 0.1844),
           (0.9421, 0.75, 0.0297),
           (0.9164, 0.25, 0.0575),
           (0.9372, 0.25, 0.1874),
           (0.8661, 0.1701, 0.0290),
           (0.1275, 0.1891, -0.0085),
           (0.9543, 0.75, 0.1459)
          ],
    spacegroup=62,
    cellpar=[8.86574, 12.6293, 8.57689, 90, 90, 90])
view(MAPbI3, viewer='ngl')

In [3]:
supercell_no_h2o=MAPbI3.repeat((2,2,2))

In [4]:
cutOff = neighborlist.natural_cutoffs(supercell_no_h2o)
nl = neighborlist.NeighborList(cutOff, self_interaction=False, bothways=True)
nl.update(supercell_no_h2o)

all_N = [atom.index for atom in supercell_no_h2o if atom.symbol == 'N']
all_H_of_N = [index for N in all_N for index in nl.get_neighbors(N)[0] if supercell_no_h2o[index].symbol == 'H'  ]
all_nh3 = all_N + all_H_of_N

In [5]:
view(supercell_no_h2o, viewer='ngl')

In [7]:
NH2O=10
nsamples=20
dmin = 1.5
dmax = 2.5
orig_h2o=molecule('H2O')
#transalte the molecule to have Oxygen in (0,0,0)
orig_h2o.translate(-1*orig_h2o.positions[0])
samples=[]
ns=0
while ns < nsamples:
    nh2o = 0
    print("Creating sample ",ns)
    supercell = supercell_no_h2o.copy() # be careful here
    while nh2o < NH2O:
        h2o = orig_h2o.copy() # be careful here
        oldcell=supercell.copy() # be careful here
        t_vector = np.random.uniform(low=-1,high=1,size=(3))
        t_vector /= np.linalg.norm(t_vector) # normalize it

        #position h2o within 1.5A---2.5A from a N atom
        t_vector *= np.random.uniform(low=dmin,high=dmax)
        a_random_N = np.random.choice(all_N)
        t_vector = supercell_no_h2o.positions[a_random_N] + t_vector
        
        #random rotation of h2o
        # you could also check scipy.spatial.transform.Rotation
        
        r_vector = R.random().as_rotvec() # vector around which to rotate and its norm is the angle (in radiants)
        h2o.rotate(180. / np.pi * np.linalg.norm(r_vector),r_vector,center=(0,0,0)) # rotate requires degrees
        
        #position h2o
        trial_h2o = h2o.copy()
        trial_h2o.translate(t_vector)
        supercell=supercell + trial_h2o
        natoms=len(supercell) 
        #O of the added h2o molecule is the third last atom: supercell[-3]
        #shortest_O_N_distances=min(supercell.get_distances(supercell[-3].index, all_N, mic=True, vector=False))
        discard=False
        for ih2o,j in product(range(natoms - 3,natoms), range(natoms-3)) :
            if supercell.get_distance(ih2o,j,mic=True,vector=False) < dmin:
                discard = True
                break
        if discard:
            supercell = oldcell.copy()
        else:
            nh2o+=1
    ns+=1
    samples.append(supercell)

Creating sample  0
Creating sample  1
Creating sample  2
Creating sample  3
Creating sample  4
Creating sample  5
Creating sample  6
Creating sample  7
Creating sample  8
Creating sample  9
Creating sample  10
Creating sample  11
Creating sample  12
Creating sample  13
Creating sample  14
Creating sample  15
Creating sample  16
Creating sample  17
Creating sample  18
Creating sample  19


In [10]:
view(samples[10], viewer='ngl')

In [11]:
v=_

In [12]:
#delete all old components
while hasattr(v.view, "component_0"):
    v.view.component_0.clear_representations()
    cid = v.view.component_0.id
    v.view.remove_component(cid)

In [13]:
v.view.add_component(nglview.ASEStructure(samples[1]), 
                     default_representation=False)
v.view.add_ball_and_stick(aspectRatio=2.0, opacity=1.0,component=0)

In [14]:
v.view.add_unitcell()
v.view.center()

In [15]:
for i,struc in enumerate(samples):
    struc.write('struc_'+str(i)+'.xyz')